Get data from github

In [ ]:
!wget https://raw.githubusercontent.com/Psellloni/bullshit/main/bert_classifier.py
!wget https://raw.githubusercontent.com/Psellloni/bullshit/main/bert_dataset.py
!wget https://raw.githubusercontent.com/Psellloni/bullshit/main/requirements.txt
!wget https://raw.githubusercontent.com/Psellloni/bullshit/main/data/pyt_dataset_1.csv
!wget https://raw.githubusercontent.com/Psellloni/bullshit/main/data/pyt_dataset_2.csv
!wget https://raw.githubusercontent.com/Psellloni/bullshit/main/data/pyt_dataset_3.csv
!wget https://raw.githubusercontent.com/Psellloni/bullshit/main/data/pyt_dataset_4.csv

Installing Packages

In [1]:
!pip install -r requirements.txt

  Using cached torch-1.11.0-cp39-none-macosx_10_9_x86_64.whl.metadata (24 kB)
  Using cached sentencepiece-0.1.95-cp39-cp39-macosx_10_6_x86_64.whl.metadata (10 kB)
  Using cached transformers-4.5.1-py3-none-any.whl.metadata (41 kB)
  Using cached pandas-1.1.5-cp39-cp39-macosx_10_9_x86_64.whl.metadata (4.7 kB)
  Using cached numpy-1.19.5-cp39-cp39-macosx_10_9_x86_64.whl.metadata (2.0 kB)
  Using cached scikit-learn-0.9.tar.gz (2.3 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [422 lines of output]
      Copying source tree into build/py3k for 2to3 transformation...
      Converting to Python3 via 2to3...
      RefactoringTool: Skipping optional fixer: buffer
      RefactoringTool: Skipping optional fixer: idioms
      RefactoringTool: Skipping optional fixer: set_literal
      RefactoringTool: Skipping optional fixer: ws_comma
      RefactoringTool: No changes to /priva

In [1]:
import pandas as pd
from sklearn.utils import shuffle

/Users/levkasenko/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/levkasenko/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
dt1 = pd.read_csv('data/pyt_dataset_1.csv')
dt2 = pd.read_csv('data/pyt_dataset_2.csv')
dt3 = pd.read_csv('data/pyt_dataset_3.csv')
dt4 = pd.read_csv('data/pyt_dataset_4.csv')

In [3]:
dt1.insert(1, "target", [0] * 1648, True)
dt2.insert(1, "target", [1] * 824, True)
dt3.insert(1, "target", [0] * 732, True)
dt4.insert(1, "target", [1] * 823, True)

In [4]:
combined_df = pd.concat([dt1, dt2])
combined_df2 = pd.concat([dt3, dt4])

In [5]:
shuffled_df = shuffle(combined_df)
shuffled_df2 = shuffle(combined_df2)

In [6]:
shuffled_df = shuffled_df.dropna()
shuffled_df2 = shuffled_df2.dropna()

In [7]:
training = shuffled_df.iloc[:int(0.9 * len(shuffled_df))]
validating = shuffled_df.iloc[int(0.9 * len(shuffled_df)):]

In [8]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

Initialize BERT classifier

In [9]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=6,
        model_save_path='/content/bert.pt'
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
classifier.load('bert.pt')

Prepare data and helpers for train and evlauation

In [14]:
classifier.preparation(
        X_train=list(training['0']),
        y_train=list(training['target']),
        X_valid=list(validating['0']),
        y_valid=list(validating['target'])
    )

Train loop

In [15]:
classifier.train()

Epoch 1/4


KeyboardInterrupt: 

Check test data

In [ ]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=6,
        model_save_path='/content/bert.pt'
)

def classify(mas):

    for img in mas:
        

In [12]:
testing = shuffled_df2.iloc[:int(0.15 * len(shuffled_df))]

In [13]:
texts = list(testing['0'])
labels = list(testing['target'])

predictions = [classifier.predict(t) for t in texts]

In [15]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.9752773268398268, recall: 0.9726398278007601, f1score: 0.9737496522714053
